# Training the V0 model

In [1]:
!pip install -r requirements.txt --quiet

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from skimage.io import imread

## Setup

In [3]:
# sagemaker
import boto3
import sagemaker
from sagemaker import get_execution_role

# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

In [4]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()

## Data

In [5]:
# iterate through S3 objects and print contents
counter = 0
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    if counter == 10:
        break
    print(obj.key)
    counter += 1

images/train/wario/wario_land_3_img_0.jpg
images/train/wario/wario_land_3_img_10.jpg
images/train/wario/wario_land_3_img_100.jpg
images/train/wario/wario_land_3_img_1000.jpg
images/train/wario/wario_land_3_img_1001.jpg
images/train/wario/wario_land_3_img_1002.jpg
images/train/wario/wario_land_3_img_1003.jpg
images/train/wario/wario_land_3_img_1004.jpg
images/train/wario/wario_land_3_img_1005.jpg
images/train/wario/wario_land_3_img_1006.jpg


In [6]:
input_data = 's3://sagemaker-eu-west-1-873555039102/images'

## Training

In [17]:
# specify an output path
prefix = 'model_v0'
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='aws', 
                    role=role,
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': 5
                    })

In [18]:
%%time 
# train the estimator on S3 training data
estimator.fit({'train': input_data})

2020-04-08 21:49:42 Starting - Starting the training job...
2020-04-08 21:49:45 Starting - Launching requested ML instances...
2020-04-08 21:50:40 Starting - Preparing the instances for training.........
2020-04-08 21:52:07 Downloading - Downloading input data......
2020-04-08 21:53:05 Training - Downloading the training image...
2020-04-08 21:53:30 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-08 21:53:31,799 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-04-08 21:53:31,826 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-04-08 21:53:31,827 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-04-08 21:53:32,082 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-04-08 21:53:32,082 sagem

KeyboardInterrupt: 

In [ ]:
estimator.model_data

## Deploying the model for inference

In [ ]:



# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.0',
                     entry_point='predict.py',
                     source_dir='source_solution')

In [ ]:
%%time
# deploy and create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [ ]:
s3://bucketname/image_folder